[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sathyasjali/EditTheGenome/blob/main/curriculum/student_materials/lesson10_blast_notebook.ipynb)

#### 🔍 What is BLAST?

**BLAST** = **B**asic **L**ocal **A**lignment **S**earch **T**ool

#### Purpose:
Find sequences in databases that are **similar** to your query sequence

#### Why Search for Similar Sequences?
1. **Infer function** - Similar sequences often have similar functions
2. **Find homologs** - Identify related genes across species
3. **Evolution** - Trace evolutionary relationships
4. **Annotation** - Identify unknown genes
5. **Species identification** - Identify organisms from DNA samples

#### Real-World Applications:
- 🦠 Identifying unknown pathogens
- 🧬 Finding disease genes
- 🌱 Crop improvement through gene discovery
- 🔬 Drug target identification

#### 📊 Key BLAST Concepts

#### 1. Query vs Subject
- **Query** = Your sequence (what you're searching with)
- **Subject** = Database sequences (what you're searching against)

#### 2. Alignment Score
- Higher score = better match
- Based on matches, mismatches, and gaps

#### 3. E-value (Expect value)
- **Most important metric!**
- Probability of finding this match by random chance
- **Lower e-value = more significant**

**E-value Guidelines:**
- `< 0.001` → Highly significant
- `0.001 - 0.01` → Significant
- `0.01 - 1` → Marginal
- `> 1` → Not significant (likely random)

#### 4. Percent Identity
- Percentage of positions that are identical
- 100% = perfect match
- >80% typically indicates homology

#### 🧪 Simulating BLAST Output

Let's create a simulated BLAST search system:

In [ ]:
import math
import random

# Simulated database of known sequences
DATABASE = {
    "Gene_A_Human": "ATGCCGATTGCAGCGGGC",
    "Gene_A_Mouse": "ATGCCGATTGCAGCGGGC",
    "Gene_A_Rat": "ATGCCGATTGCAGCGGGC",
    "Gene_B_Human": "ATGCCGATCGCAGCGGGC",
    "Gene_C_Human": "ATGCATATTGCAGCGGGC",
    "Gene_D_Yeast": "ATGATCGCTAGCGGGCTA",
    "Gene_E_Bacteria": "GCTAGCTAGCTAGCTAG C",
    "Unrelated_Seq": "TATATATATATATATATAT"
}

def calculate_alignment_score(query, subject, match=2, mismatch=-1):
    """Calculate simple alignment score."""
    min_len = min(len(query), len(subject))
    score = 0
    matches = 0
    
    for i in range(min_len):
        if query[i] == subject[i]:
            score += match
            matches += 1
        else:
            score += mismatch
    
    return score, matches, min_len

def calculate_evalue(score, query_len, db_size):
    """Simplified e-value calculation."""
    # Simplified formula (real BLAST uses more complex statistics)
    K = 0.1
    lambda_param = 0.3
    m = query_len
    n = db_size
    
    evalue = K * m * n * math.exp(-lambda_param * score)
    return evalue

print("BLAST Simulator Ready! ✓")

#### 🔎 Running a BLAST Search

In [ ]:
def blast_search(query, database, top_n=5):
    """
    Simulate a BLAST search.
    
    Args:
        query (str): Query sequence
        database (dict): Database of sequences
        top_n (int): Number of top hits to return
    
    Returns:
        list: Sorted list of hits
    """
    results = []
    db_size = sum(len(seq) for seq in database.values())
    
    for name, subject in database.items():
        score, matches, length = calculate_alignment_score(query, subject)
        identity = (matches / length) * 100 if length > 0 else 0
        evalue = calculate_evalue(score, len(query), db_size)
        
        results.append({
            'name': name,
            'score': score,
            'evalue': evalue,
            'identity': identity,
            'matches': matches,
            'length': length,
            'subject': subject
        })
    
    # Sort by score (descending)
    results.sort(key=lambda x: x['score'], reverse=True)
    
    return results[:top_n]

# Test BLAST search
query = "ATGCCGATTGCAGCGGGC"

print("BLAST Search Results")
print("=" * 80)
print(f"Query: {query}")
print(f"Query Length: {len(query)} bp\n")

hits = blast_search(query, DATABASE)

print(f"{'Rank':<6} {'Subject ID':<20} {'Score':<8} {'E-value':<12} {'Identity%':<10}")
print("=" * 80)

for rank, hit in enumerate(hits, 1):
    print(f"{rank:<6} {hit['name']:<20} {hit['score']:<8} {hit['evalue']:<12.2e} {hit['identity']:<10.1f}")

#### 📋 Detailed BLAST Report

In [ ]:
def print_detailed_report(query, hits):
    """
    Print a detailed BLAST-style report.
    """
    print("\n" + "=" * 80)
    print("DETAILED BLAST REPORT")
    print("=" * 80)
    
    for rank, hit in enumerate(hits, 1):
        print(f"\n[Hit #{rank}] {hit['name']}")
        print("-" * 80)
        print(f"  Score: {hit['score']} bits")
        print(f"  E-value: {hit['evalue']:.2e}")
        print(f"  Identities: {hit['matches']}/{hit['length']} ({hit['identity']:.1f}%)")
        
        # Show alignment
        print(f"\n  Query:   {query}")
        
        # Create match string
        matches_str = ""
        for i in range(min(len(query), len(hit['subject']))):
            if query[i] == hit['subject'][i]:
                matches_str += "|"
            else:
                matches_str += "X"
        
        print(f"           {matches_str}")
        print(f"  Subject: {hit['subject']}")
        
        # Interpretation
        if hit['evalue'] < 0.001:
            significance = "HIGHLY SIGNIFICANT"
        elif hit['evalue'] < 0.01:
            significance = "SIGNIFICANT"
        elif hit['evalue'] < 1:
            significance = "MARGINAL"
        else:
            significance = "NOT SIGNIFICANT"
        
        print(f"\n  ⭐ Significance: {significance}")
        
        if hit['identity'] >= 95:
            print(f"  💡 Interpretation: Nearly identical - likely same gene or very recent divergence")
        elif hit['identity'] >= 80:
            print(f"  💡 Interpretation: High similarity - likely homologous (related) genes")
        elif hit['identity'] >= 50:
            print(f"  💡 Interpretation: Moderate similarity - possible distant homology")
        else:
            print(f"  💡 Interpretation: Low similarity - may not be related")

# Generate detailed report
query = "ATGCCGATTGCAGCGGGC"
hits = blast_search(query, DATABASE, top_n=3)
print_detailed_report(query, hits)

#### 🎯 Practice Exercise: Interpreting E-values

In [ ]:
# Different queries with varying similarity
test_queries = {
    "Perfect Match": "ATGCCGATTGCAGCGGGC",
    "Close Match": "ATGCCGATCGCAGCGGGC",
    "Distant Match": "ATGCATATTGCAGCGGGC",
    "Poor Match": "TATATATATATATATATAT"
}

print("Comparing Different Query Qualities\n")

for query_name, query_seq in test_queries.items():
    hits = blast_search(query_seq, DATABASE, top_n=1)
    top_hit = hits[0]
    
    print(f"{query_name}:")
    print(f"  Top Hit: {top_hit['name']}")
    print(f"  E-value: {top_hit['evalue']:.2e}")
    print(f"  Identity: {top_hit['identity']:.1f}%")
    print(f"  Interpretation: ", end="")
    
    if top_hit['evalue'] < 0.001:
        print("✓ Highly reliable match")
    elif top_hit['evalue'] < 1:
        print("⚠️ Questionable match")
    else:
        print("✗ Likely random match")
    print()

#### 📊 Understanding Scoring Systems

In [ ]:
def compare_scoring_systems(query, subject):
    """
    Compare different scoring schemes.
    """
    schemes = [
        (1, 0, "Match: +1, Mismatch: 0"),
        (1, -1, "Match: +1, Mismatch: -1"),
        (2, -1, "Match: +2, Mismatch: -1"),
        (5, -4, "Match: +5, Mismatch: -4")
    ]
    
    print(f"Query:   {query}")
    print(f"Subject: {subject}\n")
    
    print("Impact of Different Scoring Schemes:")
    print("=" * 60)
    
    for match, mismatch, desc in schemes:
        score, matches, length = calculate_alignment_score(query, subject, match, mismatch)
        print(f"{desc:35} Score: {score:+4d}")

# Test
seq1 = "ATGCCGATTGCA"
seq2 = "ATGCCGATCGCA"  # One mismatch

compare_scoring_systems(seq1, seq2)

#### 🌐 Real BLAST Databases

#### Major Public Databases:

1. **NCBI BLAST** (https://blast.ncbi.nlm.nih.gov/)
   - GenBank (all sequences)
   - RefSeq (curated sequences)
   - PDB (protein structures)

2. **UniProt** (https://www.uniprot.org/)
   - Swiss-Prot (manually annotated)
   - TrEMBL (automatically annotated)

3. **Ensembl** (https://www.ensembl.org/)
   - Genomic data for vertebrates

#### Types of BLAST:
- **BLASTn** - Nucleotide vs nucleotide
- **BLASTp** - Protein vs protein
- **BLASTx** - Translated nucleotide vs protein
- **tBLASTn** - Protein vs translated nucleotide

#### 🎯 Activity: Compare Two Similar Hits

In [ ]:
# Activity: Why might two hits have similar scores but different meanings?

query = "ATGCCGATTGCAGCGGGC"

# Get top hits
all_hits = blast_search(query, DATABASE, top_n=10)

# Compare top 2-3 hits
print("Comparing Top Hits with Similar Scores\n")

for i in range(min(3, len(all_hits))):
    hit = all_hits[i]
    print(f"Hit {i+1}: {hit['name']}")
    print(f"  Score: {hit['score']}")
    print(f"  E-value: {hit['evalue']:.2e}")
    print(f"  Identity: {hit['identity']:.1f}%\n")

print("\n💭 Discussion Questions:")
print("1. Do the top hits make biological sense?")
print("2. Why might Gene_A_Human, Gene_A_Mouse, and Gene_A_Rat all have high scores?")
print("3. What does this tell us about evolution?")

#### 🤔 Reflection Questions

1. **Define e-value in one sentence:**

2. **Why search databases for similar sequences?**

3. **What does a low e-value (e.g., 1e-50) imply?**

4. **If two sequences have 95% identity, what does this suggest?**

#### 🏠 Homework

1. Visit NCBI BLAST: https://blast.ncbi.nlm.nih.gov/
2. Find and note the purpose of these databases:
   - GenBank
   - RefSeq
   - PDB
3. Look up one example BLAST result online
4. Identify the query, top hit, e-value, and percent identity

In [ ]:
# Homework coding space
# Try modifying the database and query to see how results change

# YOUR CODE HERE

#### 🎉 Summary

You've learned:
- ✅ What BLAST is and why it's essential
- ✅ How to interpret BLAST output fields
- ✅ The importance of e-values
- ✅ How scoring systems work
- ✅ Major sequence databases
- ✅ Relating sequence similarity to function

**Next lesson:** Phylogenetic trees and evolutionary relationships! 🌳